# Calculate the training accuracy from total_test file

In [32]:
import os
import pandas as pd


def total_test_acc(subj):
    total_data = []
    data_dir = r'/mnt/data/Project/DCM/BIDS/sourcedata/sub_{}/Behaviour/total_test'.format(subj)
    tottest_tmp = os.listdir(data_dir)
    tottest_list = []
    for tot in tottest_tmp:
        if '.csv' in tot:
            tottest_list.append(tot)

    for tot in tottest_list:
        data_path = os.path.join(data_dir, tot)
        print(subj,tot)
        if os.path.getsize(data_path) > 8190:
            tot_data = pd.read_csv(data_path)
            if 'apaccuracy' in tot_data.columns:
                if 'dpaccuracy' in tot_data.columns:
                    tot_acc = tot_data.dropna(subset=['apaccuracy','dpaccuracy'],how='all')
                    tot_acc = tot_acc[['姓名','date','apaccuracy','dpaccuracy']]
                    total_data.append(tot_acc)
                else:
                    tot_acc = tot_data.dropna(subset=['apaccuracy'],how='all')
                    tot_acc = tot_acc[['姓名','date','apaccuracy']]
                    total_data.append(tot_acc)
            elif 'dpaccuracy' in tot_data.columns:
                    tot_acc = tot_data.dropna(subset=['dpaccuracy'],how='all')
                    tot_acc = tot_acc[['姓名','date','dpaccuracy']]
                    total_data.append(tot_acc)
            else:
                print("The {} do not have acc info".format(tot)) 
    return total_data


total_data = [] 
subjests = [str(i).zfill(3) for i in range(29,50)]
for subj in subjests:
    sub_data = total_test_acc(subj)
    if isinstance(sub_data, list):
        total_data.extend(sub_data)
        
total_data = pd.concat(total_data,axis=0)

i = 1
save_path = r'/mnt/data/Project/DCM/BIDS/derivatives/behaviour/result/training/{}_total_data.csv'.format(i)
while os.path.exists(save_path):
    i += 1
    save_path = r'/mnt/data/Project/DCM/BIDS/derivatives/behaviour/result/training/{}_total_data.csv'.format(i)
    
total_data.to_csv(save_path)
total_data

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Project/DCM/BIDS/sourcedata/sub_029/Behaviour/total_test'

# Calculate the training accuracy from game2 training file

In [1]:
import os
import pandas as pd


def game2trian_acc(subj):
    data_dir = r'/mnt/workdir/DCM/sourcedata/sub_{}/Behaviour/fmri_task-game2-train'.format(subj)
    game2train_tmp = os.listdir(data_dir)
    game2train_list = []
    for g2t in game2train_tmp:
        if '.csv' in g2t:
            game2train_list.append(g2t)
            
    print(game2train_list)
    train_data_blocked = [] 
    for g2t in game2train_list:
        data_path = os.path.join(data_dir, g2t)
        print(subj,g2t)

        if os.path.getsize(data_path) > 12190:
            game2data = pd.read_csv(data_path)
            train_data = game2data.dropna(subset=['train_accuracy'])
            data_len = len(train_data)
            block_num = int(data_len/32)
            for i in range(block_num):
                print('block',i+1)
                if i < (block_num-1):
                    block_data = train_data[i*32:(i+1)*32].copy()    
                else:
                    block_data = train_data[i*32:].copy()

                # 假设这个block是攻击力
                block_ap_diff = block_data['ap_diff']
                block_dp_diff = block_data['dp_diff']

                corrAns_ap = [1 if a < 0 else 2 for a in block_ap_diff]
                corrAns_dp = [1 if a < 0 else 2 for a in block_dp_diff]
                corrAns = block_data['correctAns'].to_list()

                if corrAns == corrAns_ap:
                    block_data.loc[:,'dim'] = 'ap'
                elif corrAns == corrAns_dp:
                    block_data.loc[:,'dim'] = 'dp'
                else:
                    print("The code is wrong! ")

                train_data_blocked.append(block_data.iloc[-1,:][['姓名','participant','date','dim','train_accuracy']])

    return train_data_blocked
    

game2train_result = [] 
subjests = [str(i).zfill(3) for i in range(74,79)]
for subj in subjests:
    sub_result = game2trian_acc(subj)
    if isinstance(sub_result, list):
        game2train_result.extend(sub_result)
        
i = 1
save_path = r'/mnt/workdir/DCM/BIDS/derivatives/behaviour/result/training/{}_game2train_result.csv'.format(i)
while os.path.exists(save_path):
    i += 1
    save_path = r'/mnt/workdir/DCM/BIDS/derivatives/behaviour/result/training/{}_game2train_result.csv'.format(i)

game2train_result = pd.DataFrame(game2train_result)  
game2train_result.to_csv(save_path)
game2train_result

['Z65_task-game2train_2022_Apr_17_1245.csv']
074 Z65_task-game2train_2022_Apr_17_1245.csv
block 1
block 2
block 3
block 4
['B22_task-game2train_2022_Apr_17_1524.csv', 'B22_task-game2train_2022_Apr_17_1445.csv']
075 B22_task-game2train_2022_Apr_17_1524.csv
block 1
block 2
075 B22_task-game2train_2022_Apr_17_1445.csv
block 1
block 2
block 3
block 4
['Z10_task-game2train_2022_Apr_24_1453.csv']
076 Z10_task-game2train_2022_Apr_24_1453.csv
block 1
block 2
block 3
block 4
['Z67_task-game2train_2022_Apr_24_1256.csv']
077 Z67_task-game2train_2022_Apr_24_1256.csv
block 1
block 2
block 3
block 4
['Z13_task-game2train_2022_Apr_24_1556.csv']
078 Z13_task-game2train_2022_Apr_24_1556.csv
block 1
block 2
block 3
block 4


,姓名,participant,date,dim,train_accuracy
64,王旭茹,Z65,2022_Apr_17_1245,ap,0.844
128,王旭茹,Z65,2022_Apr_17_1245,dp,0.875
192,王旭茹,Z65,2022_Apr_17_1245,dp,1.000
256,王旭茹,Z65,2022_Apr_17_1245,ap,0.938
64,刘悦天,B22,2022_Apr_17_1524,ap,0.938
128,刘悦天,B22,2022_Apr_17_1524,dp,0.844
64,liuyuetian,B22,2022_Apr_17_1445,ap,0.781
128,liuyuetian,B22,2022_Apr_17_1445,dp,0.844
192,liuyuetian,B22,2022_Apr_17_1445,dp,0.875
256,liuyuetian,B22,2022_Apr_17_1445,ap,0.719
